In [ ]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm

In [ ]:
embs = pd.read_csv('database/collins/collins_tadw.csv').set_index('id')
emb_np = embs.to_numpy()

In [ ]:
network = nx.Graph()
with open('database/collins/collins.txt') as f:
  edges = f.readlines()
for line in edges:
  x,y = line.split()
  network.add_edge(x,y)


pro_to_id = {}
i = 0
for n in network.nodes:
    pro_to_id[n] = i
    i= i+1
    
pro_to_id['YDL007W']

In [ ]:
deg_sum = 0
for n in tqdm(network.nodes):
    deg_sum += network.degree[n] 
avg = deg_sum/len(network.nodes)

hub_count = 0
for n in tqdm(network.nodes):
    if network.degree[n] > avg:
        hub_count += 1
hub_count
        

In [ ]:
from math import*
 
def square_rooted(x):
 
   return round(sqrt(sum([a*a for a in x])),3)
 
def cosine_similarity(x,y):
 
 numerator = sum(a*b for a,b in zip(x,y))
 denominator = square_rooted(x)*square_rooted(y)
 return round(numerator/float(denominator),3)

In [ ]:
W = np.zeros((len(network.nodes),len(network.nodes)))
nodes = list(network.nodes)
for e in tqdm(network.edges):
    n1 = e[0]
    i = nodes.index(n1)
    n2 = e[1]
    j = nodes.index(n2)
    W[i][j] = cosine_similarity(emb_np[i],emb_np[j])
    network[n1][n2]['weight'] = cosine_similarity(emb_np[i],emb_np[j])
#W

In [ ]:
# degree matrix
import math 
#W = sim_matrix
for i in range(W.shape[0]):
    W[i][i] = 0

D = np.diag(np.sum(np.array(W), axis=1))
L = D-W
L_p = np.array(nx.normalized_laplacian_matrix(network, nodelist=None, weight='weight').todense())


In [ ]:
#file1=open("Form_CYC20083.txt")
#print(file1)
reference_complex=[]
#reference_num=len(file1.readlines())
with open('database/Form_CYC20083.txt') as f:
    ji = f.readlines()
    reference_num=len(f.readlines())
    for j in ji:
        j = j.rstrip()
        j = j.rstrip('\n')
        complex_list = j.split('\t')
        reference_complex.append(complex_list)
#file1.close()
reference_complex[-1]
#reference_num

In [ ]:
actual_complex =[]
for com in reference_complex:
    x = []  
    for p in com:
        if p in pro_to_id.keys():
            x.append(pro_to_id[p])
    actual_complex.append(x)
actual_complex

In [ ]:
index = []
reference_complex=[]
for i in range(len(actual_complex)):
    if len(actual_complex[i]) > 0:
        index.append(i)
        reference_complex.append(actual_complex[i])
print(len(reference_complex))
print(len(index))




In [ ]:
def positive_predictive_value(reference, predicted):
    num, den = 0., 0.
    for cluster in predicted:
        isects = [len(set(cluster).intersection(set(compl))) for compl in reference]
        isects.append(0.)
        num += max(isects)
        den += sum(isects)
    if den == 0.:
        return 0.
    return num / den
#positive_predictive_value(actual_clusters, predicted_clusters)
def clusteringwise_sensitivity(reference, predicted):
    num, den = 0., 0.
    for complx in reference:
        den += len(complx)
        num += max(len(set(complx).intersection(set(cluster))) for cluster in predicted)
    if den == 0.:
        return 0.
    return num / den

def accuracy(reference, predicted):
    return (clusteringwise_sensitivity(reference, predicted) * \
            positive_predictive_value(reference, predicted)) ** 0.5
#print(accuracy(actual_complex, predicted_complex))
#print(clusteringwise_sensitivity(actual_clusters, predicted_clusters))

In [ ]:
k =100
e, v = np.linalg.eig(L_p)
idx = np.argsort(e)[:k]
eigen_vectors = np.zeros((len(network.nodes),k))
for i in range(len(idx)):
    eigen_vectors[:,i] = v[:,idx[i]]
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=31, random_state=0).fit(eigen_vectors)
labels = kmeans.labels_
    
predicted_complex = []
for i in range(k):
    predicted_complex.append([])
for i in range(len(labels)):
    predicted_complex[labels[i]].append(i)
        
number=0
c_number=0
row=1
for i in predicted_complex:
    overlapscore=0.0
    for j in reference_complex:
        set1 = set(i)
        set2 = set(j)
        overlap = set1 & set2
        if float((len(set1) * len(set2))) != 0:
            score = float((pow(len(overlap), 2))) / float((len(set1) * len(set2)))
        if (score > overlapscore):
            overlapscore = score
    if (overlapscore > 0.2):
            number = number + 1
    row=row+1


for i in tqdm(reference_complex):
    overlapscore=0.0
    for j in predicted_complex:
        set1 = set(i)
        set2 = set(j)
        overlap = set1 & set2
        if float((len(set1) * len(set2))) != 0:
            score = float((pow(len(overlap), 2))) / float((len(set1) * len(set2)))
        if(score > overlapscore):
            overlapscore = score
    if(overlapscore > 0.2):
        c_number=c_number+1
precision=float(number/len(predicted_complx))
print('precision: ', precision)
recall=float(c_number/float(len(reference_complex)))
print('recall: ', recall)
F1=float((2*precision*recall)/(precision+recall))
print('F1: ',F1)
print('acc: ',accuracy(actual_complex, predicted_complex))